# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 6 2023 1:08PM,254473,19,60019624,"GUSA Granules USA, Inc.",Released
1,Jan 6 2023 12:26PM,254471,10,PRF-44162-BO,Bio-PRF,Released
2,Jan 6 2023 12:25PM,254470,10,PRF-45893-BO,Bio-PRF,Released
3,Jan 6 2023 12:25PM,254470,10,PRF-46114,Bio-PRF,Released
4,Jan 6 2023 12:25PM,254470,10,PRF-46118,Bio-PRF,Released
5,Jan 6 2023 12:25PM,254470,10,PRF-46121,Bio-PRF,Released
6,Jan 6 2023 12:25PM,254470,10,PRF-46125,Bio-PRF,Released
7,Jan 6 2023 12:25PM,254470,10,PRF-46128,Bio-PRF,Released
8,Jan 6 2023 12:25PM,254470,10,PRF-46133,Bio-PRF,Released
9,Jan 6 2023 12:25PM,254470,10,PRF-46141,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
37,254468,Released,23
38,254469,Executing,1
39,254470,Released,17
40,254471,Released,1
41,254473,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254468,NaN,NaN,23.0
254469,NaN,1.0,NaN
254470,NaN,NaN,17.0
254471,NaN,NaN,1.0
254473,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254367,2.0,8.0,20.0
254377,1.0,0.0,3.0
254388,0.0,0.0,1.0
254389,0.0,1.0,1.0
254390,0.0,0.0,5.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254367,2,8,20
254377,1,0,3
254388,0,0,1
254389,0,1,1
254390,0,0,5


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254367,2,8,20
1,254377,1,0,3
2,254388,0,0,1
3,254389,0,1,1
4,254390,0,0,5


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254367,2,8,20
1,254377,1,,3
2,254388,,,1
3,254389,,1,1
4,254390,,,5


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 6 2023 1:08PM,254473,19,"GUSA Granules USA, Inc."
1,Jan 6 2023 12:26PM,254471,10,Bio-PRF
2,Jan 6 2023 12:25PM,254470,10,Bio-PRF
19,Jan 6 2023 12:16PM,254469,12,Hush Hush
20,Jan 6 2023 12:10PM,254468,10,ISDIN Corporation
43,Jan 6 2023 12:04PM,254467,10,"Snap Medical Industries, LLC"
44,Jan 6 2023 12:03PM,254466,21,"NBTY Global, Inc."
45,Jan 6 2023 11:59AM,254465,15,"Mizner Bioscience, LLC"
48,Jan 6 2023 11:57AM,254464,10,ISDIN Corporation
53,Jan 6 2023 11:55AM,254463,16,Sartorius Bioprocess Solutions


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 6 2023 1:08PM,254473,19,"GUSA Granules USA, Inc.",,,1
1,Jan 6 2023 12:26PM,254471,10,Bio-PRF,,,1
2,Jan 6 2023 12:25PM,254470,10,Bio-PRF,,,17
3,Jan 6 2023 12:16PM,254469,12,Hush Hush,,1,
4,Jan 6 2023 12:10PM,254468,10,ISDIN Corporation,,,23
5,Jan 6 2023 12:04PM,254467,10,"Snap Medical Industries, LLC",,,1
6,Jan 6 2023 12:03PM,254466,21,"NBTY Global, Inc.",,,1
7,Jan 6 2023 11:59AM,254465,15,"Mizner Bioscience, LLC",,,3
8,Jan 6 2023 11:57AM,254464,10,ISDIN Corporation,,,5
9,Jan 6 2023 11:55AM,254463,16,Sartorius Bioprocess Solutions,,1,3


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 6 2023 1:08PM,254473,19,"GUSA Granules USA, Inc.",1,,
1,Jan 6 2023 12:26PM,254471,10,Bio-PRF,1,,
2,Jan 6 2023 12:25PM,254470,10,Bio-PRF,17,,
3,Jan 6 2023 12:16PM,254469,12,Hush Hush,,1,
4,Jan 6 2023 12:10PM,254468,10,ISDIN Corporation,23,,
5,Jan 6 2023 12:04PM,254467,10,"Snap Medical Industries, LLC",1,,
6,Jan 6 2023 12:03PM,254466,21,"NBTY Global, Inc.",1,,
7,Jan 6 2023 11:59AM,254465,15,"Mizner Bioscience, LLC",3,,
8,Jan 6 2023 11:57AM,254464,10,ISDIN Corporation,5,,
9,Jan 6 2023 11:55AM,254463,16,Sartorius Bioprocess Solutions,3,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 6 2023 1:08PM,254473,19,"GUSA Granules USA, Inc.",1,,
1,Jan 6 2023 12:26PM,254471,10,Bio-PRF,1,,
2,Jan 6 2023 12:25PM,254470,10,Bio-PRF,17,,
3,Jan 6 2023 12:16PM,254469,12,Hush Hush,,1,
4,Jan 6 2023 12:10PM,254468,10,ISDIN Corporation,23,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 6 2023 1:08PM,254473,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
1,Jan 6 2023 12:26PM,254471,10,Bio-PRF,1.0,NaN,NaN
2,Jan 6 2023 12:25PM,254470,10,Bio-PRF,17.0,NaN,NaN
3,Jan 6 2023 12:16PM,254469,12,Hush Hush,NaN,1.0,NaN
4,Jan 6 2023 12:10PM,254468,10,ISDIN Corporation,23.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 6 2023 1:08PM,254473,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
1,Jan 6 2023 12:26PM,254471,10,Bio-PRF,1.0,0.0,0.0
2,Jan 6 2023 12:25PM,254470,10,Bio-PRF,17.0,0.0,0.0
3,Jan 6 2023 12:16PM,254469,12,Hush Hush,0.0,1.0,0.0
4,Jan 6 2023 12:10PM,254468,10,ISDIN Corporation,23.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3816436,186.0,8.0,3.0
12,254469,0.0,1.0,0.0
15,1017794,8.0,1.0,0.0
16,1017717,6.0,1.0,0.0
18,254420,1.0,0.0,0.0
19,1526626,19.0,1.0,0.0
20,763305,3.0,0.0,0.0
21,508864,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3816436,186.0,8.0,3.0
1,12,254469,0.0,1.0,0.0
2,15,1017794,8.0,1.0,0.0
3,16,1017717,6.0,1.0,0.0
4,18,254420,1.0,0.0,0.0
5,19,1526626,19.0,1.0,0.0
6,20,763305,3.0,0.0,0.0
7,21,508864,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,186.0,8.0,3.0
1,12,0.0,1.0,0.0
2,15,8.0,1.0,0.0
3,16,6.0,1.0,0.0
4,18,1.0,0.0,0.0
5,19,19.0,1.0,0.0
6,20,3.0,0.0,0.0
7,21,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,186.0
1,12,Released,0.0
2,15,Released,8.0
3,16,Released,6.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21
Status,,,,,,,,
Completed,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,8.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0
Released,186.0,0.0,8.0,6.0,1.0,19.0,3.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21
0,Completed,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,8.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0
2,Released,186.0,0.0,8.0,6.0,1.0,19.0,3.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21
0,Completed,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,8.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0
2,Released,186.0,0.0,8.0,6.0,1.0,19.0,3.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()